# Clustering AVR-Pia
---

This notebook will guide you through the process of clustering 99 complexes of the AVR-Pia protein against a sample of rice proteins. All of the models are dimeric. The 99 models originate from a proteome-wide screen of AVR-Pia against the rice proteome (O. sativa subsp. japonica, 43,000+ initial models). To select this sample we first ran AlphaCRV on all the models, and the 99 structures are part of some of the best clusters that were identified. Running the pipeline on these models should be enough to reproduce the results and give you an idea of the workflow.

# Prerequisites

- Install AlphaCRV on a conda environment and activate it
- Download the models and sequences for this example from [Zenodo](https://zenodo.org/record/5525340/files/alphafold-multimer.tar.gz?download=1)

# 1. Cluster the models with the `clustering` command

Run the following command to cluster the models. Make sure to change the paths according to your system:

```bash
alphacrv-cluster \
  --bait ./examples/AVRPia/AVRPia.fasta \
  --binders ./examples/AVRPia/AVRPia_binders.fasta \
  --models_dir ./examples/AVRPia/AVRPia_vs_rice_models \
  --destination ./examples/AVRPia/AVRPia_vs_rice_clusters \
  --cpus 8
```

After collecting the quality scores from the models in `--models_dir`, it will count how many models are there with an ipTM scores higher than the threshold (0.75 by default). It will prompt you to confirm or to modify the threshold. After that, it will proceed with the clustering.

The full output is as follows:

After this step you will have a directory with the following structure:

Now let's look at some of the important files:

In [1]:
from pathlib import Path
import pandas as pd

In [2]:
results_dir = Path('./AVRPia/AVRPia_vs_rice_clusters/')

## See clusters

The `merged_clusters.csv` file contains the list of models with their corresponding sequence, structure and merged clusters. It also has the quality scores provided by AlphaFold.

In [3]:
clusters = pd.read_csv(results_dir / 'merged_clusters/merged_clusters.csv')

In [4]:
clusters

,complex,str_rep,seq_rep,merged_rep,member,iptm,iptm+ptm
0,6Q76-1_Q5JN27-1,Q0IYV8.pdb_B,C7IWS6,Q6Z1A9.pdb_B,Q5JN27,0.946405,0.880667
1,6Q76-1_Q6K9S6-1,A0A0P0WCR3.pdb_B,Q6K9S6,Q6Z1A9.pdb_B,Q6K9S6,0.934689,0.859638
2,6Q76-1_A0A0P0VEX0-1,Q0IYV8.pdb_B,A0A0P0XZP1,Q6Z1A9.pdb_B,A0A0P0VEX0,0.933272,0.916800
3,6Q76-1_Q2R3L3-1,A0A0P0VF33.pdb_B,A0A0P0Y2U8,Q6Z1A9.pdb_B,Q2R3L3,0.926249,0.909282
4,6Q76-1_A0A0P0UZA6-1,A0A0P0UZA6.pdb_B,A0A0P0UZA6,Q7XL74.pdb_B,A0A0P0UZA6,0.921295,0.827867
...,...,...,...,...,...,...,...
94,6Q76-1_A0A0P0XDF8-1,Q0IYV8.pdb_B,A0A0N7KMC1,Q6Z1A9.pdb_B,A0A0P0XDF8,0.768701,0.751252
95,6Q76-1_A0A0P0Y3K7-1,Q0IYV8.pdb_B,A0A0P0Y3K7,Q6Z1A9.pdb_B,A0A0P0Y3K7,0.766905,0.758089
96,6Q76-1_Q0IYI4-1,Q7XL74.pdb_B,Q7XL74,Q7XL74.pdb_B,Q0IYI4,0.766464,0.718768
97,6Q76-1_Q6Z0A9-1,Q0IYV8.pdb_B,A0A0N7KMC1,Q6Z1A9.pdb_B,Q6Z0A9,0.764813,0.740763


The columns of the `clusters` DataFrame are:
- `complex`: The name of the complex. This is the same name as the directory where the model is stored.
- `str_rep`: Name of the structure cluster representative
- `seq_rep`: Name of the sequence cluster representative
- `merged_rep`: Name of the merged cluster representative (sequence + structure)
- `member`: The ID of the binder protein
- `iptm`: The ipTM score of the model
- `iptm+ptm`: The ipTM+PTM score of the model (it is calculated by AlphaFold as 0.8*ipTM + 0.2*pTM)

See the number of different merged clusters:

In [5]:
clusters.merged_rep.unique().shape

(3,)

For this example, the models and sequences from the 99 binder proteins were summarized in 3 clusters. Much fewer structures to sort through!

## See alignment scores

Alignment scores are calcualted for each cluster by aligning all vs all members of the cluster.

In [6]:
alignment_scores = pd.read_csv(results_dir / 'merged_clusters/alignment_scores.csv')

In [7]:
alignment_scores.head()

,cluster,ref,member,tmscore_ref,tmscore_m,aligned_length,rmsd
0,Q6Z1A9.pdb_B,Q5JN27,Q6K9S6,0.22351,0.56157,206,12.72
1,Q6Z1A9.pdb_B,Q5JN27,A0A0P0VEX0,0.53580,0.44881,564,20.63
2,Q6Z1A9.pdb_B,Q5JN27,Q2R3L3,0.32890,0.66186,274,10.16
3,Q6Z1A9.pdb_B,Q5JN27,A0A0P0Y2U8,0.23944,0.51955,225,11.05
4,Q6Z1A9.pdb_B,Q5JN27,C7IWS6,0.55887,0.81560,393,9.61


The columns of the `alignment_scores` DataFrame are:
- `cluster`: The name of the cluster
- `ref`: Binder ID of the reference structure in the alignment
- `member`: Binder ID of the second structure in the alignment
- `tmscore_ref`: TM-score based on the reference structure
- `tmscore_m`: TM-score based on the second structure
- `aligned_length`: Length of the alignment
- `rmsd`: RMSD of the alignment

Based on these scores, the median scores are calculated for each cluster member to find the best representative of the cluster (the one with lowest RMSD score to the other members).

## Read median scores and find top clusters

Now we can rank the clusters based on the median alignment scores of the cluster representatives:

In [8]:
median_scores = pd.read_csv(results_dir / 'merged_clusters/median_scores.csv')

In [9]:
median_scores.shape

(99, 7)

The `median_scores` DataFrame contains the median alignment scores of each cluster member when aligned to all other members of the same cluster.

In [10]:
median_scores.head()

,cluster,member,tmscore,rmsd,aligned_length,cluster_size,fraction_binder
0,P42211.pdb_B,A0A0P0VAB8,0.56928,21.55,419.0,10.0,0.862408
1,P42211.pdb_B,A0A0P0VAW6,0.59934,12.81,407.0,10.0,0.867008
2,P42211.pdb_B,A0A0P0WVN9,0.39349,24.73,412.0,10.0,0.577181
3,P42211.pdb_B,P42211,0.26818,20.91,363.0,10.0,0.707434
4,P42211.pdb_B,Q0D5V1,0.66389,10.75,402.0,10.0,0.902703


The columns of the `median_scores` DataFrame are:
- `cluster`: The name of the cluster
- `member`: ID of the cluster member (binder molecule)
- `tmscore`: Median TM-score of the complex against all other complexes in this cluster
- `rmsd`: Median RMSD of the complex against all other complexes in this cluster
- `aligned_length`: Median length of the alignment
- `cluster_size`
- `fraction_binder`: In average, how much of the binder molecule is included in the alignments of this complex against all other complexes (calculated as `(aligned_length - bait_length)/binder_length`). This is just meant to be an approximation of how complete the aligmnents are for this cluster member.

The next step is to select the cluster representatives. For this, we first need to filter out the cluster members with poor quality alignments, according to the following criteria:
- Small size
- Low median TM-score
- High median RMSD
- Low fraction of the binder aligned in the cluster representative

In [11]:
# Select the clusters with the following criteria:
select = ((median_scores.cluster_size >= 5) & \
            (median_scores.tmscore >= 0.2) & \
            (median_scores.fraction_binder >= 0.2) & \
            (median_scores.rmsd <= 15))
median_scores_filtered = median_scores[select]

In [12]:
median_scores_filtered.shape

(83, 7)

See how many clusters are left after filtering:

In [13]:
median_scores_filtered.cluster.unique().shape

(3,)

Function to format tables:

In [14]:
import seaborn as sns
cm_r = sns.color_palette("mako_r", as_cmap=True)
cm = sns.color_palette("mako", as_cmap=True)

In [15]:
def make_pretty(styler):
    styler.format(precision=2)
    styler.background_gradient(axis=0, cmap=cm_r, subset=pd.IndexSlice[:,"cluster_size"],vmin=5,vmax=15)
    styler.background_gradient(axis=0, cmap=cm_r, subset=pd.IndexSlice[:,"tmscore"],vmin=0.2,vmax=0.8)
    styler.background_gradient(axis=0, cmap=cm, subset=pd.IndexSlice[:,"rmsd"],vmin=2,vmax=10)
    styler.background_gradient(axis=0, cmap=cm_r, subset=pd.IndexSlice[:,"fraction_binder"],vmin=0.2,vmax=0.9)
    return styler

## RESULT 1: See clusters ranked by RMSD

Finally, we can rank the clusters and see which ones have a good combination of low RMSD and large cluster size. These ones are the most likely to contain the true binder.

In [16]:
# Select the rows with the minimum RMSD for each cluster
select = median_scores_filtered.groupby('cluster').rmsd.idxmin()
columns = ['cluster', 'tmscore', 'rmsd', 'cluster_size', 'fraction_binder']
median_scores_filtered.loc[select,columns].sort_values(by='rmsd').style.pipe(make_pretty)

,cluster,tmscore,rmsd,cluster_size,fraction_binder
59,Q7XL74.pdb_B,0.90,1.49,40.00,0.94
22,Q6Z1A9.pdb_B,0.55,10.65,49.00,0.84
4,P42211.pdb_B,0.66,10.75,10.00,0.90


Here we can see that the cluster `Q7XL74.pdb_B` has the lowest median RMSD and the highest median TM-score. It also has a very large size with 40 members.

## RESULT 2: See clusters ranked by size

In [17]:
# Select the rows with the minimum RMSD for each cluster
select = median_scores_filtered.groupby('cluster').rmsd.idxmin()
columns = ['cluster', 'tmscore', 'rmsd', 'cluster_size', 'fraction_binder']
median_scores_filtered.loc[select, columns].sort_values(by='cluster_size', ascending=False).style.pipe(make_pretty)

,cluster,tmscore,rmsd,cluster_size,fraction_binder
22,Q6Z1A9.pdb_B,0.55,10.65,49.00,0.84
59,Q7XL74.pdb_B,0.90,1.49,40.00,0.94
4,P42211.pdb_B,0.66,10.75,10.00,0.90


Here we can see that the cluster `Q6Z1A9.pdb_B` is the largest cluster with 49 members, but it has a much worse RMSD than `Q7XL74.pdb_B`. It would be interesting to look at both and see how they compare. So we managed to reduce more than 43,000 starting models to only two promising clusters!

For this example we know that in our list of candidate binders there are 14 homologues of the true binder protein. We can find out which clusters contain these homologues:

In [25]:
homologues = ['Q6YY33', 'Q6YY34', 'A0A0P0VKX7', 'Q0JCK8', 'Q6YY31', 'Q7XJV3',
       'A0A0N7KFK3', 'Q7XJV0', 'A0A0P0WB87', 'Q6EPT4', 'Q0J314', 'Q6EPT2',
       'Q8S5W0', 'Q2QSQ7']

In [26]:
clusters[clusters.member.isin(homologues)][['complex','merged_rep','iptm','iptm+ptm']]

,complex,merged_rep,iptm,iptm+ptm
6,6Q76-1_Q6EPT2-1,Q7XL74.pdb_B,0.913843,0.847186
14,6Q76-1_Q6EPT4-1,Q7XL74.pdb_B,0.903790,0.819935
26,6Q76-1_Q2QSQ7-1,Q7XL74.pdb_B,0.892084,0.806683
30,6Q76-1_Q0J314-1,Q7XL74.pdb_B,0.886320,0.820720
38,6Q76-1_Q8S5W0-1,Q7XL74.pdb_B,0.875703,0.805274
43,6Q76-1_Q6YY34-1,Q7XL74.pdb_B,0.867315,0.839418
47,6Q76-1_Q7XJV3-1,Q7XL74.pdb_B,0.862152,0.831752
48,6Q76-1_Q6YY33-1,Q7XL74.pdb_B,0.860912,0.830552
55,6Q76-1_A0A0N7KFK3-1,Q7XL74.pdb_B,0.857097,0.819887
57,6Q76-1_A0A0P0VKX7-1,Q7XL74.pdb_B,0.854087,0.827703


They are all in the top cluster by RMSD!

# 2. Make pymol sessions for the top clusters with `make_pymol_sessions`

Run the following command to select the top clusters that we saw above, make pymol sessions of the top clusters, and optionally do structural clustering on each cluster to find subclusters:

```bash
alphacrv-rank \
  --clusters_dir ./examples/AVRPia/AVRPia_vs_rice_clusters \
  --min_members 5 \
  --min_tmscore 0.2 \
  --max_rmsd 15 \
  --cluster_clusters
```

The program will show you the top clusters that will be used to make the pymol sessions. You can press `Enter` to continue, or exit the program with `Ctrl+C` to change the filtering parameters.

This command should create the following files in the `./examples/AVRPia/AVRPia_vs_rice_clusters/merged_clusters/` / `merged_clusters` directory:

- `clustered_clusters.csv`: Contains the subclusters for each of the top clusters.
- `cluster_<cluster_ID>/`: Contains the PDBs of each cluster, and a PyMol session with the cluster members.
- `cluster_<cluster_ID>_clusters/`: Contains the results of the `foldseek easy-cluster` run on the cluster members.

## Read clustered clusters

The following DataFrame contains the subclusters for each of the top clusters:

In [27]:
clustered_clusters = pd.read_csv(results_dir / 'merged_clusters/clustered_clusters.csv')

In [28]:
clustered_clusters.head()

,subcluster_rep,member,cluster
0,P42211.pdb_B,P42211,P42211.pdb_B
1,P42211.pdb_B,Q6F4N5,P42211.pdb_B
2,P42211.pdb_B,A0A0P0VAB8,P42211.pdb_B
3,P42211.pdb_B,Q0D5V1,P42211.pdb_B
4,P42211.pdb_B,Q8LNM1,P42211.pdb_B


Now we can look at the most interesting clusters and their subclusters in more detail:

## RESULT 1: Cluster Q7XL74.pdb_B (contains true binder homologs, top cluster by size)

In [29]:
cluster = 'Q7XL74.pdb_B'

See subclusters:

In [30]:
clustered_clusters[clustered_clusters.cluster==cluster]

,subcluster_rep,member,cluster
59,A0A0P0UZA6.pdb_B,A0A0P0UZA6,Q7XL74.pdb_B
60,Q7XL74.pdb_B,Q7XL74,Q7XL74.pdb_B
61,Q7XL74.pdb_B,Q0IYI4,Q7XL74.pdb_B
62,Q7XL74.pdb_B,Q75J02,Q7XL74.pdb_B
63,Q7XL74.pdb_B,A0A0P0WEB4,Q7XL74.pdb_B
64,Q7XL74.pdb_B,A0A0P0VRA1,Q7XL74.pdb_B
65,Q7XL74.pdb_B,Q0JBG2,Q7XL74.pdb_B
66,Q7XL74.pdb_B,A0A0P0XD00,Q7XL74.pdb_B
67,Q7XL74.pdb_B,A0A0P0V126,Q7XL74.pdb_B
68,Q7XL74.pdb_B,A3AYA2,Q7XL74.pdb_B


Here we have two subclusters (`subcluster_rep` column). However, one of them (`A0A0P0UZA6.pdb_B`) has only one member, and if we look at it in the PyMOL session we can see that it is very similar to the other complexes. The only difference is the HMA domain being shorter. So for our purposes it would be ok to take them all together.

We can see in the `median_scores` DataFrame the model that has the best alignments to all other structures (best representative of the cluster).

In [31]:
median_scores[median_scores.cluster==cluster].sort_values(by='rmsd').head(10)

,cluster,member,tmscore,rmsd,aligned_length,cluster_size,fraction_binder
59,Q7XL74.pdb_B,A0A0N7KFK3,0.89660,1.49,141.0,40.0,0.935897
61,Q7XL74.pdb_B,A0A0P0V0T5,0.89782,1.50,140.0,40.0,0.960000
93,Q7XL74.pdb_B,Q84TS2,0.90749,1.52,140.0,40.0,0.972973
62,Q7XL74.pdb_B,A0A0P0V126,0.90529,1.58,140.0,40.0,0.972973
90,Q7XL74.pdb_B,Q7XJV3,0.90317,1.62,141.0,40.0,0.948052
84,Q7XL74.pdb_B,Q6YY31,0.90490,1.62,141.0,40.0,0.948052
63,Q7XL74.pdb_B,A0A0P0VKX7,0.87889,1.74,142.0,40.0,0.936709
85,Q7XL74.pdb_B,Q6YY33,0.88542,1.74,142.0,40.0,0.948718
70,Q7XL74.pdb_B,A2ZV04,0.89069,1.74,138.0,40.0,0.972222
86,Q7XL74.pdb_B,Q6YY34,0.88160,1.82,142.0,40.0,0.936709


We can also select the IDs of the binder proteins in this cluster, and run them through a tool such as DAVID to perform enrichment analysis.

In [32]:
members = clusters[clusters.merged_rep==cluster].member

In [33]:
for m in members:
    print(m)

A0A0P0UZA6
Q6EPT2
Q8LN41
Q6EPT4
Q6I5G4
Q7XL74
Q84TB9
Q2QSQ7
A0A0P0WEB4
Q0J314
A2ZV04
Q5JL91
Q8S5W0
Q8LNN6
Q75J02
Q6YY34
B7E663
Q7XJV3
Q6YY33
Q8LMS1
A0A0P0XD00
Q7G2B2
A0A0N7KFK3
A0A0P0V0T5
A0A0P0VKX7
A0A0P0XFR7
A0A0P0Y5Q0
A0A0P0WB87
Q10KH9
A0A0P0V126
A3ADD6
Q6YY31
A3AYA2
Q0JBG2
Q0JCK8
Q8S181
A0A0P0VRA1
Q7XJV0
Q0IYI4
Q84TS2


Looking at the `examples/AVRPia/AVRPia_vs_rice_clusters/merged_clusters/cluster_Q7XL74.pdb_B/session.pse` file in PyMol, we can visualize the cluster and produce a figure like this:

<img src='./pictures/AVRPia_cluster.png' height='600px'>

When aligning the HMA domains of the binder structures (grey), the binding of the AVR-Pia protein seems pretty consistent across most of the models in the cluster.

Tip: In PyMol, you can remove the residues with low pLDDT score to make the figure cleaner, with the command:
    
```
select b < 50; remove sele
```